In [59]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.datasets import CSVClassificationDataset

In [16]:
import pandas as pd

In [131]:
df = pd.read_csv("./data/TrainNER.csv", sep=";", encoding = "ISO-8859-1")
df.fillna('', inplace=True)

In [132]:
def fill_sentence_numbers(df):
    sentences = df["Sentence #"].values
    current = sentences[0]
    new_sentence_nums = []
    for s in sentences:
        if s != '':
            current = s
        new_sentence_nums.append(int(current.replace("Sentence: ", "")))
    df["Sentence #"] = new_sentence_nums
        
fill_sentence_numbers(df)

In [133]:
df = df[df["Word"] != ""]
df

,Sentence #,Word,POS,Tag
0,1,President,NNP,B-per
1,1,Karzai,NNP,I-per
2,1,thanked,VBD,O
3,1,his,PRP$,O
4,1,allies,NNS,O
...,...,...,...,...
196640,9000,prices,NNS,O
196641,9000,and,CC,O
196642,9000,foreign,JJ,O
196643,9000,investment,NN,O


In [134]:
train = df[df["Sentence #"] <= 6000]
dev = df[(df["Sentence #"] > 6000) & (df["Sentence #"] >= 7500)]
test = df[df["Sentence #"] > 7500]

def write_in_proper_format(df, filename):
    prev_sentence_num = df.iloc[0]["Sentence #"]
    
    for i in range(len(df)):
        row = df.iloc[i]
        current_sentence_num = row["Sentence #"]
        
        with open(filename, "a") as file:
            if current_sentence_num != prev_sentence_num:
                file.write("\n")
            file.write(row["Word"])
            file.write(" ")
            file.write(row["POS"])
            file.write(" ")
            file.write(row["Tag"])
            file.write("\n")
        prev_sentence_num = current_sentence_num

output_csv_train = "./data/train.txt"
output_csv_dev = "./data/dev.txt"
output_csv_test = "./data/test.txt"
write_in_proper_format(train, output_csv_train)
write_in_proper_format(dev, output_csv_dev)
write_in_proper_format(test, output_csv_test)
    

In [ ]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    # GloVe embeddings
    WordEmbeddings('glove'),

    # contextual string embeddings, forward
    PooledFlairEmbeddings('news-forward', pooling='min'),

    # contextual string embeddings, backward
    PooledFlairEmbeddings('news-backward', pooling='min'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type)

# initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/szwb-ner',
              train_with_dev=True,  
              max_epochs=150,
                )

2020-09-30 20:10:40,980 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to C:\Users\Marci\AppData\Local\Temp\tmpt2_hym2s


100%|██████████| 160000128/160000128 [03:44<00:00, 712776.32B/s]

2020-09-30 20:14:25,726 copying C:\Users\Marci\AppData\Local\Temp\tmpt2_hym2s to cache at C:\Users\Marci\.flair\embeddings\glove.gensim.vectors.npy


2020-09-30 20:14:26,432 removing temp file C:\Users\Marci\AppData\Local\Temp\tmpt2_hym2s
2020-09-30 20:14:26,815 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to C:\Users\Marci\AppData\Local\Temp\tmpaysghglf


100%|██████████| 21494764/21494764 [00:30<00:00, 706419.64B/s]

2020-09-30 20:14:57,463 copying C:\Users\Marci\AppData\Local\Temp\tmpaysghglf to cache at C:\Users\Marci\.flair\embeddings\glove.gensim
2020-09-30 20:14:57,495 removing temp file C:\Users\Marci\AppData\Local\Temp\tmpaysghglf


2020-09-30 20:14:58,698 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to C:\Users\Marci\AppData\Local\Temp\tmpg9go46w4


100%|██████████| 73034575/73034575 [01:43<00:00, 703974.34B/s]

2020-09-30 20:16:42,584 copying C:\Users\Marci\AppData\Local\Temp\tmpg9go46w4 to cache at C:\Users\Marci\.flair\embeddings\news-backward-0.4.1.pt


2020-09-30 20:16:42,692 removing temp file C:\Users\Marci\AppData\Local\Temp\tmpg9go46w4
2020-09-30 20:16:43,670 ----------------------------------------------------------------------------------------------------
2020-09-30 20:16:43,672 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
    )
    (list_embedding_2): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias